## ANN Classifier
for code to make the classifier click [here](https://www.kaggle.com/code/yashasgaonkar/ann-classifier/edit/run/216103405)

In [38]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle as pkl
from tensorflow.keras.models import load_model


In [39]:
### load the trained model,scaled pkl,onehotenc
model = load_model('model.h5')

with open('scaler.pkl', 'rb') as f:
    scaler = pkl.load(f)

with open('onehot_encoder_geography.pkl', 'rb') as f:
    onehotenc = pkl.load(f)

with open('label_encoder_gender.pkl', 'rb') as f:
    labelenc = pkl.load(f)


d:\Documents\Desktop\backend\genai\.venv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Documents\Desktop\backend\genai\.venv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Documents\Desktop\backend\genai\.venv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when usi

In [40]:
input_data = {
    'CreditScore': 619,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 42,
    'Tenure': 2,
    'Balance': 0.00,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 101348.88
}

In [43]:
## One hot encode Geography
geo_encoded = onehotenc.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehotenc.categories_[0])


d:\Documents\Desktop\backend\genai\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names


In [44]:
##Combine the input data with the one hot encoded geography
input_df = pd.DataFrame([input_data])
input_df = pd.concat([input_df.reset_index(drop=True), geo_encoded_df], axis=1)
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,619,France,Female,42,2,0.0,1,1,1,101348.88,1.0,0.0,0.0


In [45]:
input_df['Gender'] = labelenc.transform(input_df['Gender'])
input_df = input_df.drop(columns=['Geography'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,619,0,42,2,0.0,1,1,1,101348.88,1.0,0.0,0.0


In [48]:
## Error observed due to my inconsistency in the column names (used .categories_[0] instead)
input_df.rename(columns={'France': 'Geography_France', 'Germany': 'Geography_Germany', 'Spain': 'Geography_Spain'}, inplace=True)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.0,1,1,1,101348.88,1.0,0.0,0.0


In [49]:
input_scaler = scaler.transform(input_df)
input_scaler

array([[-0.33880827, -1.09499335,  0.29493847, -1.04241787, -1.21847056,
        -0.91668767,  0.64920267,  0.97481699,  0.01595384,  1.00150113,
        -0.57946723, -0.57638802]])

In [51]:
prediction = model.predict(input_scaler)
if prediction > 0.5:
    print('Customer will exit the bank')
else:
    print('Customer will stay with the bank')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
Customer will stay with the bank
